# Graded Lab: Agentic Workflows

In this lab, you will build an agentic system that generates a short research report through planning, external tool usage, and feedback integration. Your workflow will involve:

### Agents

* **Planning Agent / Writer**: Creates an outline and coordinates tasks.
* **Research Agent**: Gathers external information using tools like Arxiv, Tavily, and Wikipedia.
* **Editor Agent**: Reflects on the report and provides suggestions for improvement.


### Research Tools

By importing `research_tools`, you gain access to several search utilities:

- `research_tools.arxiv_search_tool(query)` → search academic papers from **arXiv**  

  *Example:* `research_tools.arxiv_search_tool("neural networks for climate modeling")`

- `research_tools.tavily_search_tool(query)` → perform web searches with the **Tavily API**  

  *Example:* `research_tools.tavily_search_tool("latest trends in sunglasses fashion")`

- `research_tools.wikipedia_search_tool(query)` → retrieve summaries from **Wikipedia**  

  *Example:* `research_tools.wikipedia_search_tool("Ensemble Kalman Filter")`

Run the cell below to make them available.

In [1]:
# =========================
# Imports
# =========================

# --- Standard library 
from datetime import datetime
import re
import json
import ast


# --- Third-party ---
from IPython.display import Markdown, display
from aisuite import Client

# --- Local / project ---
import research_tools

In [2]:
import unittests

### Initialize client

Create a shared client instance for upcoming calls.

In [3]:
CLIENT = Client()

## Exercise 1: planner_agent

### Objective
Correctly set up a call to a language model (LLM) to generate a research plan.

### Instructions

1. **Focus Areas**:
   - Ensure `CLIENT.chat.completions.create` is correctly configured.
   - Pass the `model` and `messages` parameters correctly:
     - **Model**: Use `"openai:o4-mini"` by default.
     - **Messages**: Set with `{"role": "user", "content": user_prompt}`.
     - **Temperature**: Fixed at 1 for creative outputs.

### Notes

- The prompt is pre-defined and guides the LLM on task requirements.
- Only return a formatted list of steps — no extra text.

Focus on the LLM call setup to complete the task.

In [4]:
# GRADED FUNCTION: planner_agent

def planner_agent(topic: str, model: str = "openai:o4-mini") -> list[str]:
    """
    Generates a plan as a Python list of steps (strings) for a research workflow.

    Args:
        topic (str): Research topic to investigate.
        model (str): Language model to use.

    Returns:
        List[str]: A list of executable step strings.
    """

    
    # Build the user prompt
    user_prompt = f"""
    You are a planning agent responsible for organizing a research workflow with multiple intelligent agents.

    🧠 Available agents:
    - A research agent who can search the web, Wikipedia, and arXiv.
    - A writer agent who can draft research summaries.
    - An editor agent who can reflect and revise the drafts.

    🎯 Your job is to write a clear, step-by-step research plan **as a valid Python list**, where each step is a string.
    Each step should be atomic, executable, and must rely only on the capabilities of the above agents.

    🚫 DO NOT include irrelevant tasks like "create CSV", "set up a repo", "install packages", etc.
    ✅ DO include real research-related tasks (e.g., search, summarize, draft, revise).
    ✅ DO assume tool use is available.
    ✅ DO NOT include explanation text — return ONLY the Python list.
    ✅ The final step should be to generate a Markdown document containing the complete research report.

    Topic: "{topic}"
    """

    # Add the user prompt to the messages list
    messages = [{"role": "user", "content": user_prompt}]

    ### START CODE HERE ###

    # Call the LLM
    response = CLIENT.chat.completions.create( 
        # Pass in the model
        model=model,
        # Define the messages. Remember this is meant to be a user prompt!
        messages=messages,
        # Keep responses creative
        temperature=1, 
    )

    ### END CODE HERE ###

    # Extract message from response
    steps_str = response.choices[0].message.content.strip()

    # Parse steps
    steps = ast.literal_eval(steps_str)

    return steps

In [5]:
# Test your code!
unittests.test_planner_agent(planner_agent)

 All tests passed!


## Exercise 2: research_agent

### Objective
Set up a call to a language model (LLM) to perform a research task using various tools.

### Instructions

**Focus Areas**:

- **Creating a Custom Prompt**:
  - **Define the Role**: Clearly specify the role, such as "research assistant."
  - **List Available Tools** (as strings inside the prompt, not the actual functions):
    - Use `arxiv_tool` to find academic papers.
    - Use `tavily_tool` for general web searches.
    - Use `wikipedia_tool` for accessing encyclopedic knowledge.
  - **Specify the Task**: Include a placeholder in your prompt for defining the specific task that needs to be accomplished.
  - **Include Date Information**: Add a placeholder for the current date or time to provide context.

- **Creating Messages Dict**:
  - Ensure the `messages` are correctly set with `{"role": "user", "content": prompt}`.

- **Creating Tools List**:
  - Create a list of tools for use, such as `research_tools.arxiv_search_tool`, `research_tools.tavily_search_tool`, and `research_tools.wikipedia_search_tool`.

- **Correctly Setting the Call to the LLM**:
  - Pass the `model`, `messages`, and `tools` parameters accurately.
  - Set `tool_choice` to `"auto"` for automatic tool selection.
  - Limit interactions with `max_turns=6`.

### Notes

- The function provides pre-coded blocks where you need to replace placeholder values.
- The approach allows the LLM to use tools dynamically based on the task.

Focus on accurately setting the messages, tools, and LLM call parameters to complete the task.

In [6]:
# GRADED FUNCTION: research_agent

def research_agent(task: str, model: str = "openai:gpt-4o", return_messages: bool = False):
    """
    Executes a research task using tools via aisuite (no manual loop).
    Returns either the assistant text, or (text, messages) if return_messages=True.
    """
    print("==================================")  
    print("🔍 Research Agent")                 
    print("==================================")

    current_time = datetime.now().strftime('%Y-%m-%d')
    
    ### START CODE HERE ###

    # Create a customizable prompt by defining the role (e.g., "research assistant"),
    # listing tools (arxiv_tool, tavily_tool, wikipedia_tool) for various searches,
    # specifying the task with a placeholder, and including a current_time placeholder.
    prompt = f"""
    You are a research assistant.
    You can use the following tools when helpful:
    - arxiv_tool: find academic papers and preprints on arXiv.
    - tavily_tool: perform general web searches for recent or broad information.
    - wikipedia_tool: access encyclopedic knowledge and summaries.

    Task to complete: {task}
    Current date: {current_time}

    Produce a concise, well-cited research result. Use tools when needed and include links.
    """
    
    # Create the messages dict to pass to the LLM. Remember this is a user prompt!
    messages = [{"role": "user", "content": prompt}]

    # Save all of your available tools in the tools list. These can be found in the research_tools module.
    # You can identify each tool in your list like this: 
    # research_tools.<name_of_tool>, where <name_of_tool> is replaced with the function name of the tool.
    tools = [
        research_tools.arxiv_search_tool,
        research_tools.tavily_search_tool,
        research_tools.wikipedia_search_tool,
    ]
    
    # Call the model with tools enabled
    response = CLIENT.chat.completions.create(  
        # Set the model
        model=model,
        # Pass in the messages. You already defined this!
        messages=messages,
        # Pass in the tools list. You already defined this!
        tools=tools,
        # Set the LLM to automatically choose the tools
        tool_choice="auto",
        # Set the max turns to 6
        max_turns=6
    )  
    
    ### END CODE HERE ###

    content = response.choices[0].message.content
    print("✅ Output:\n", content)

    
    return (content, messages) if return_messages else content  

In [7]:
# Test your code!
unittests.test_research_agent(research_agent)

🔍 Research Agent
✅ Output:
 To provide you with three key references, I will search for recent academic papers on arXiv, explore general information using Tavily, and look into encyclopedic knowledge on Wikipedia. Please specify the research topic you want these references to focus on.
🔍 Research Agent
✅ Output:
 The paper "Attention Is All You Need," published in 2017, introduces the Transformer model, a novel architecture for sequence-to-sequence tasks such as language translation. This model eliminates the need for recurrence or convolution by relying solely on self-attention mechanisms, allowing it to capture complex relationships within sequential data efficiently. The Transformer model achieved state-of-the-art results in multiple translation tasks, and its introduction marked a significant advancement in the field of deep learning by demonstrating the power of attention mechanisms ([Medium Summary](https://medium.com/@dminhk/attention-is-all-you-need-summary-6f0437e63a91)). The 

## Exercise 3: writer_agent

### Objective
Set up a call to a language model (LLM) for executing writing tasks like drafting, expanding, or summarizing text.

### Instructions

1. **Focus Areas**:
   - **System Prompt**:
     - Define `system_prompt` to assign the LLM the role of a writing agent focused on generating academic or technical content.
   - **System and User Messages**:
     - Create `system_msg` using `{"role": "system", "content": system_prompt}`.
     - Create `user_msg` using `{"role": "user", "content": task}`.
   - **Messages List**:
     - Combine `system_msg` and `user_msg` into a `messages` list.

### Notes

- The function is designed to produce well-structured text by setting the correct prompts.
- Temperature is set to 1.0 to allow for creative variance in the writing outputs.

Ensure the system prompt and messages are defined properly to achieve a structured output from the LLM.

In [8]:
# GRADED FUNCTION: writer_agent
def writer_agent(task: str, model: str = "openai:gpt-4o") -> str: # @REPLACE def writer_agent(task: str, model: str = None) -> str:
    """
    Executes writing tasks, such as drafting, expanding, or summarizing text.
    """
    print("==================================")
    print("✍️ Writer Agent")
    print("==================================")

    ### START CODE HERE ###
    
    # Create the system prompt.
    # This should assign the LLM the role of a writing agent specialized in generating well-structured academic or technical content
    system_prompt = (
        "You are a writing agent specialized in generating well-structured, clear, and academically rigorous "
        "technical or research content. Follow formal tone, use proper structure (e.g., introduction, main points, conclusion), "
        "and ensure the text is cohesive and logically organized."
    )

    # System message (system_msg) → sets the overall role and behavior of the model.
    # User message (user_msg) → gives the specific task or question to solve.
    # examples: system_msg: “You are a helpful math tutor.”, user_msg: “Explain fractions to a 10-year-old.”
    
    # Define the system msg by using the system_prompt and assigning the role of system
    system_msg = {"role": "system", "content": system_prompt}

    # Define the user msg. In this case the user prompt should be the task passed to the function
    user_msg = {"role": "user", "content": task}

    # Add both system and user messages to the messages list
    messages = [system_msg, user_msg]
    
    ### END CODE HERE ###

    response = CLIENT.chat.completions.create(
        model=model, 
        messages=messages,
        temperature=1.0
    )

    return response.choices[0].message.content

In [9]:
# Test your code!
unittests.test_writer_agent(writer_agent)

✍️ Writer Agent
 All tests passed!


## Exercise 4: editor_agent

### Objective
Configure a call to a language model (LLM) to perform editorial tasks such as reflecting, critiquing, or revising drafts.

### Instructions

1. **Focus Areas**:
   - **System Prompt**:
     - Define `system_prompt` to assign the LLM the role of an editor agent whose task is to reflect on, critique, or improve drafts.
   - **System and User Messages**:
     - Create `system_msg` using `{"role": "system", "content": system_prompt}`.
     - Create `user_msg` using `{"role": "user", "content": task}`.
   - **Messages List**:
     - Combine `system_msg` and `user_msg` into a `messages` list.

### Notes

- The editor agent is tailored for enhancing the quality of text by setting an appropriate role and task in the prompts.
- Temperature is set to 0.7, balancing creativity and coherence in editorial outputs.

Ensure the system prompt and messages are accurately set up to perform effective editorial tasks with the LLM.

In [10]:
# GRADED FUNCTION: editor_agent
def editor_agent(task: str, model: str = "openai:gpt-4o") -> str:
    """
    Executes editorial tasks such as reflection, critique, or revision.
    """
    print("==================================")
    print("🧠 Editor Agent")
    print("==================================")
    
    ### START CODE HERE ###

    # Create the system prompt.
    # This should assign the LLM the role of an editor agent specialized in reflecting on, critiquing, or improving existing drafts.
    system_prompt = (
        "You are an editor agent. Your job is to reflect on, critique, and improve drafts. "
        "Provide thoughtful, structured feedback and, when needed, revised versions of the text. "
        "Focus on clarity, coherence, style, and overall quality."
    )
    
    # Define the system msg by using the system_prompt and assigning the role of system
    system_msg = {"role": "system", "content": system_prompt}
    
    # Define the user msg. In this case the user prompt should be the task passed to the function
    user_msg = {"role": "user", "content": task}
    
    # Add both system and user messages to the messages list
    messages = [system_msg, user_msg]
    
    ### END CODE HERE ###
    
    response = CLIENT.chat.completions.create(
        model=model, 
        messages=messages,
        temperature=0.7 
    )
    
    return response.choices[0].message.content

In [11]:
# Test your code!
unittests.test_editor_agent(editor_agent)

🧠 Editor Agent
 All tests passed!


### 🎯 The Executor Agent

The `executor_agent` manages the workflow by executing each step of a given plan. It:

1. Decides **which agent** (`research_agent`, `writer_agent`, or `editor_agent`) should handle the step.
2. Builds context from the outputs of previous steps.
3. Sends the enriched task to the selected agent.
4. Collects and stores the results in a shared history.

👉 **Do not implement or modify this function.** It is already provided as the orchestration component of the multi-agent pipeline.

Notice that `planner_agent` might return a long list of steps. Because of this, the maximum number of steps is set to a maximum of 4 to keep running time reasonable.

In [12]:
agent_registry = {
    "research_agent": research_agent,
    "editor_agent": editor_agent,
    "writer_agent": writer_agent,
}

def clean_json_block(raw: str) -> str:
    """
    Clean the contents of a JSON block that may come wrapped with Markdown backticks.
    """
    raw = raw.strip()
    if raw.startswith("```"):
        raw = re.sub(r"^```(?:json)?\n?", "", raw)
        raw = re.sub(r"\n?```$", "", raw)
    return raw.strip()

In [13]:
def executor_agent(topic, model: str = "openai:gpt-4o", limit_steps: bool = True):

    plan_steps = planner_agent(topic)
    max_steps = 4

    if limit_steps:
        plan_steps = plan_steps[:min(len(plan_steps), max_steps)]
    
    history = []

    print("==================================")
    print("🎯 Editor Agent")
    print("==================================")

    for i, step in enumerate(plan_steps):

        agent_decision_prompt = f"""
        You are an execution manager for a multi-agent research team.

        Given the following instruction, identify which agent should perform it and extract the clean task.

        Return only a valid JSON object with two keys:
        - "agent": one of ["research_agent", "editor_agent", "writer_agent"]
        - "task": a string with the instruction that the agent should follow

        Only respond with a valid JSON object. Do not include explanations or markdown formatting.

        Instruction: "{step}"
        """
        response = CLIENT.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": agent_decision_prompt}],
            temperature=0,
        )

        raw_content = response.choices[0].message.content
        cleaned_json = clean_json_block(raw_content)
        agent_info = json.loads(cleaned_json)

        agent_name = agent_info["agent"]
        task = agent_info["task"]

        context = "\n".join([
            f"Step {j+1} executed by {a}:\n{r}" 
            for j, (s, a, r) in enumerate(history)
        ])
        enriched_task = f"""
        You are {agent_name}.

        Here is the context of what has been done so far:
        {context}

        Your next task is:
        {task}
        """

        print(f"\n🛠️ Executing with agent: `{agent_name}` on task: {task}")

        if agent_name in agent_registry:
            output = agent_registry[agent_name](enriched_task)
            history.append((step, agent_name, output))
        else:
            output = f"⚠️ Unknown agent: {agent_name}"
            history.append((step, agent_name, output))

        print(f"✅ Output:\n{output}")

    return history

In [14]:
# If you want to see the full workflow without limiting the number of steps. Set limit_steps to False
# Keep in mind this could take more than 10 minutes to complete
executor_history = executor_agent("The ensemble Kalman filter for time series forecasting", limit_steps=True)

md = executor_history[-1][-1].strip("`")  
display(Markdown(md))

🎯 Editor Agent

🛠️ Executing with agent: `research_agent` on task: Search arXiv and Google Scholar for foundational and recent papers on the ensemble Kalman filter for time series forecasting
🔍 Research Agent
✅ Output:
 I encountered a timeout issue while trying to retrieve data from arXiv, but I successfully obtained relevant research articles from general web sources. Below is a summary of key findings on the ensemble Kalman filter applied to time series forecasting:

1. **Forecasting Electricity Prices Using Ensemble Kalman Filter**:
   - This study explores the use of the Ensemble Kalman Filter (EnKF) for forecasting electricity prices, focusing on the efficacy of the method by evaluating its performance using statistical measures like MAPE and RMSE.
   - [Read more here](https://www.scienpress.com/Upload/JSEM/Vol%209_1_2.pdf).

2. **Ensemble Kalman Filter for GAN-ConvLSTM Based Long Lead-Time Forecasting**:
   - This article discusses integrating EnKF with machine learning models,

✅ Output:
 The Ensemble Kalman Filter (EnKF) has been extensively applied in time series forecasting, incorporating diverse methodological approaches and performance metrics. Here's a summary of the approaches and metrics used from both recent research and foundational descriptions:

### Methodological Approaches:

1. **Non-Parametric Methods**:
   - Some studies ([APS](https://link.aps.org/doi/10.1103/PhysRevX.6.011021)) explore model-free filtering through Kalman's equations and data-driven Takens' methods, facilitating accurate forecasts even without explicit models.

2. **Integration with Machine Learning Models**:
   - Research integrates EnKF with machine learning frameworks such as GAN-ConvLSTM ([ScienceDirect](https://www.sciencedirect.com/science/article/pii/S1877750323000844)) to handle the uncertainties in long lead-time and chaotic systems. This hybrid approach is aimed at reducing forecast uncertainty and correcting model errors in real-time.

3. **Data Assimilation and Pe

### Literature Review Outline on the Ensemble Kalman Filter in Time Series Forecasting

#### **I. Introduction to the Ensemble Kalman Filter (EnKF)**
- **Overview**: The Ensemble Kalman Filter (EnKF) is a recursive filter designed for problems with numerous variables, often used in geophysical models ([Wikipedia](https://en.wikipedia.org/wiki/Ensemble_Kalman_filter)).
- **Applications**: EnKF is crucial for data assimilation in ensemble forecasting, and it is efficient for systems assumed to have Gaussian distributions.

#### **II. Themes in Recent Research**

1. **Integration with Machine Learning Models**
   - **Hybrid Models**: Research often integrates EnKF with models like GAN and ConvLSTM to improve long-term forecast accuracy and reduce uncertainty ([Gan-ConvLSTM EnKF Study](https://www.sciencedirect.com/science/article/pii/S1877750323000844)).

2. **Non-Parametric and Model-Free Methods**
   - **Kalman-Takens Approach**: Utilizing model-free strategies helps in scenarios where explicit models are unavailable ([Slud, UMD](https://www.math.umd.edu/~slud/RITF17/enkf-tutorial.pdf), [APS](https://link.aps.org/doi/10.1103/PhysRevX.6.011021)).

3. **Data Assimilation and Perturbed Observations**
   - EnKF is applied extensively in weather forecasting via perturbed observations to manage vast data and improve accuracy ([Yihua Yang Study](http://arxiv.org/pdf/2004.04275v2)).

4. **Preserving Invariants in Observation Systems**
   - **Invariant Preservation**: Techniques to maintain critical invariants in predictions are explored ([Mathieu Le Provost Study](http://arxiv.org/pdf/2404.14328v1)).

#### **III. Identified Gaps and Challenges**

1. **Handling of Multiscale Phenomena**
   - *Gap*: Localization challenges in high-resolution applications, integration with variational approaches, and satellite data assimilation remain underexplored ([Journals Ametsoc](https://journals.ametsoc.org/view/journals/mwre/144/12/mwr-d-15-0440.1.xml)).

2. **Scalability with Ensemble Size**
   - *Gap*: The effect of ensemble size on error estimation and computational efficiency in larger systems ([UMD Math](https://www.math.umd.edu/~slud/RITF17/enkf-tutorial.pdf)).

#### **IV. Open Questions**

1. **Optimal Ensemble Size and Composition**
   - What is the balance between ensemble size and forecast accuracy in high-dimensional spaces?
  
2. **Integration of Non-Gaussian Distributions**
   - How can EnKF be adapted for non-Gaussian settings while retaining efficiency?

3. **Combining EnKF with Other Data Assimilation Techniques**
   - What are the most effective approaches to hybridize EnKF with variational methods for better model accuracy?

#### **V. Conclusion**

- **Summary of Findings**: EnKF remains a versatile and powerful tool for time series forecasting. Its integration with machine learning and novel non-parametric approaches opens up innovative research paths.
- **Future Directions**: Addressing gaps and open questions will require interdisciplinary collaboration, particularly in enhancing computational techniques and exploring hybrid frameworks.

This outline consolidates the current state of research on the Ensemble Kalman Filter, identifying key themes, gaps, and future research directions within the domain of time series forecasting.